# Model Tuning

### 기존에 학습한 pretrained 모델을 로드한 후, 일부 layer의 weight를 freeze하고 나머지 layer는 제거하고 새롭게 layer를 추가해서 모델을 구성해본다.

In [1]:
##tenssorflow==2.3 환경에서 구현 
import tensorflow as tf
import numpy as np
import copy

#기존에 학습한 전체 model 로드
from tensorflow.keras.models import load_model
model =load_model("./mnist_classification_model.h5")

#로드한 모델 layer 정보 확인
print("layer 수:",len(model.layers))
model.summary()

#layer freeze 여부 확인 (True>trainable, False>freeze)
for i,layer in enumerate(model.layers):
    print("layer number: %s"%(i+1),layer) 
    print("Trainable:",layer.trainable)

layer 수: 20
Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 27, 27, 16)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 27, 27, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 26, 26

In [2]:
#첫번째 layer부터 열세번째 layer까지만 weight를 freeze
for i,layer in enumerate(model.layers):
    if i==13:
        break
    model.layers[i].trainable=False
    
#weight가 freeze된 layer를 제외하고 나머지 layer 제거
layers=model.layers[:13]

#10개 layer를 새롭게 추가하여 수정된 모델 생성 
input_layer=layers[0].output
x=layers[-1].output
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, activation='relu',padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(2, 2))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Conv2D(kernel_size=(3,3), filters=256, activation='relu',padding="same")(x)
x = tf.keras.layers.MaxPooling2D((2,2),strides=(2, 2))(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
new_model = tf.keras.Model(inputs=input_layer,outputs=outputs)

In [3]:
#layer 수가 증가되었음을 확인
print("layer 수:",len(new_model.layers))
new_model.summary()

layer 수: 23
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 27, 27, 16)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 27, 27, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 26, 26, 32)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 26, 26

In [4]:
# 수정된 모델의 열세번째 layer까지 weight가 freeze 되었음을 확인
for i,layer in enumerate(new_model.layers):
    print("layer number: %s"%(i+1),layer) 
    print("Trainable:",layer.trainable)

layer number: 1 <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f596695dc88>
Trainable: False
layer number: 2 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f59669212b0>
Trainable: False
layer number: 3 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f59669217f0>
Trainable: False
layer number: 4 <tensorflow.python.keras.layers.core.Dropout object at 0x7f59669219e8>
Trainable: False
layer number: 5 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f5966921ba8>
Trainable: False
layer number: 6 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f5966921eb8>
Trainable: False
layer number: 7 <tensorflow.python.keras.layers.core.Dropout object at 0x7f59668e10f0>
Trainable: False
layer number: 8 <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f59668e12b0>
Trainable: False
layer number: 9 <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f59668e15c0>
Trainable: False
layer 